20163228 남유선

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd

# Read Files

In [2]:
#
# Read Train File	
#
size_row    = 28    # height of the image
size_col    = 28    # width of the image

train_file  = "mnist_train.csv"

handle_file = open(train_file, "r")
train_data  = handle_file.readlines()
handle_file.close()

train_num   = len(train_data)

train_list  = np.zeros((train_num, size_row * size_col), dtype=float)
train_a     = np.zeros((train_num, size_row * size_col + 1), dtype=float)
train_label = np.zeros((10, train_num), dtype=int)
train_original_label = np.zeros(train_num, dtype=int)

count = 0
label = 2
for line in train_data :
    line_data = line.split(',')
    train_original_label[count] = line_data[0]
    
    for i in range (10) :
        if (line_data[0] == str(i)) :
            train_label[i, count] = 1
        else :
            train_label[i, count] = -1
    
    # Image Vector
    im_vector = np.asfarray(line_data[1:])
    train_list[count, :] = im_vector
    
    train_a[count, 0]    = 1
    train_a[count, 1:]   = im_vector
    count += 1

In [3]:
#
# Read Test File
#
test_file   = "mnist_test.csv"

handle_file = open(test_file, "r")
test_data   = handle_file.readlines()
handle_file.close()

test_num    = len(test_data)

test_list   = np.zeros((test_num, size_row * size_col), dtype=float)
test_a      = np.zeros((test_num, size_row * size_col + 1), dtype=float)
test_label = np.zeros((10, test_num), dtype=int)
test_original_label = np.zeros(test_num, dtype=int)

count = 0
label = -2
for line in test_data :
    line_data = line.split(',')
    test_original_label[count] = line_data[0]
    
    for i in range (10) :
        if (line_data[0] == str(i)) :
            test_label[i, count] = 1
        else :
            test_label[i, count] = -1
    
    im_vector = np.asfarray(line_data[1:])
    test_list[count, :] = im_vector
    
    test_a[count, 0]    = 1
    test_a[count, 1:]   = im_vector
    count += 1

# Calculate Pseudo Inverse

In [4]:
class LeastSquare() :
    
    def __init__(self, image, label) :
        self.image = image
        self.label = label
        self.n     = len(label)
        self.m     = len(image[0])
        
        
    def calculate_ata(self) :
        self.trans_image = self.image.T
        self.ata = np.dot(self.trans_image, self.image)
        
        
    def calculate_atb(self) :
        re_label = self.label.reshape(self.n, 1)
        self.atb = np.dot(self.trans_image, re_label)
        
        
    def calculate_theta(self) :
        mati_ata = np.linalg.pinv(self.ata)
        mat_atb = np.asmatrix(self.atb)
        self.aia = np.dot(mati_ata, mat_atb)
        
        
    def train(self) :
        self.calculate_ata()
        self.calculate_atb()
        
        self.calculate_theta()
        
        return self.aia

In [5]:
theta = np.zeros((10, size_row * size_col + 1), dtype=float)
for i in range (10) :
    ls       = LeastSquare(train_a, train_label[i, :])
    setaa    = ls.train()
    theta[i] = setaa.reshape(785)

# Calculate y and label according to argmax

In [6]:
train_y = np.zeros((train_num, 10), dtype=float)
test_y  = np.zeros((test_num, 10) , dtype=float)

for i in range (10) :
    train_y[:, i] = np.dot(train_a, theta[i])
    test_y[: , i] = np.dot(test_a , theta[i])

In [7]:
train_pred = np.zeros(train_num, dtype=int)
test_pred  = np.zeros(test_num,  dtype=int)

for i in range (train_num) :
    train_pred[i] = train_y[i, :].argmax()
    
for i in range (test_num) :
    test_pred[i] = test_y[i, :].argmax()

# Calculate Accuracy

## Train Data

In [8]:
train_result = confusion_matrix(train_original_label, train_pred) 
print(train_result)

[[5682    7   18   14   24   43   64    4   61    6]
 [   2 6548   40   15   19   31   14   12   55    6]
 [  99  264 4792  149  108   11  234   91  192   18]
 [  42  167  176 5158   32  125   56  115  135  125]
 [  10   99   42    6 5212   50   39   23   59  302]
 [ 164   95   28  432  105 3991  192   36  235  143]
 [ 108   74   61    1   70   90 5476    0   35    3]
 [  55  189   37   47  170    9    2 5426   10  320]
 [  75  493   63  226  105  221   56   20 4412  180]
 [  68   60   20  117  371   12    4  492   38 4767]]


In [9]:
# Calculate true positive rate & error rate
train_tp  = 0
train_err = 0
for i in range (10) :
    train_tp += train_result[i][i]
    
train_tp /= train_num
train_err = 1 - train_tp

In [10]:
print("Train Data")
print("True Positive Rate : ", train_tp)
print("Error Rate         : ", train_err)

Train Data
True Positive Rate :  0.8577333333333333
Error Rate         :  0.14226666666666665


## Test Data

In [11]:
test_result  = confusion_matrix(test_original_label , test_pred)
print(test_result)

[[ 944    0    1    2    2    7   14    2    7    1]
 [   0 1107    2    2    3    1    5    1   14    0]
 [  18   54  813   26   15    0   42   22   37    5]
 [   4   17   23  880    5   17    9   21   22   12]
 [   0   22    6    1  881    5   10    2   11   44]
 [  23   18    3   72   24  659   23   14   39   17]
 [  18   10    9    0   22   17  875    0    7    0]
 [   5   40   16    6   26    0    1  884    0   50]
 [  14   46   11   30   27   40   15   12  759   20]
 [  15   11    2   17   80    1    1   77    4  801]]


In [12]:
# Calculate true positive rate & error rate
test_tp  = 0
test_err = 0
for i in range (10) :
    test_tp += test_result[i][i]
    
test_tp /= test_num
test_err = 1 - test_tp

In [13]:
print("Test Data")
print("True Positive Rate : ", test_tp)
print("Error Rate         : ", test_err)

Test Data
True Positive Rate :  0.8603
Error Rate         :  0.13970000000000005
